# Creation of shipping mode table


In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 351a858d-7f20-4e0a-a6af-8edc478c2b84, 3, Finished, Available, Finished)

In [ ]:
# Creat a shipmode table

In [2]:
%%sql
create table if not exists ecom_proj_LH.gold_shipmode
(
    ShipMode_ID Long,
    Ship_Mode string,
    Created_TS timestamp,
    Modified_TS timestamp
)

StatementMeta(, 351a858d-7f20-4e0a-a6af-8edc478c2b84, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Max_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from ecom_proj_LH.gold_shipmode").first()[0]

StatementMeta(, 351a858d-7f20-4e0a-a6af-8edc478c2b84, 5, Finished, Available, Finished)

In [4]:
Max_Date

StatementMeta(, 351a858d-7f20-4e0a-a6af-8edc478c2b84, 6, Finished, Available, Finished)

'1900-01-01'

In [ ]:
#Now we pull the ship_mode column from bronze table

In [6]:
df_bronze=spark.sql("select distinct Ship_Mode from ecom_proj_LH.bronze_sales where Modified_TS > '{}'".format(Max_Date))

StatementMeta(, 351a858d-7f20-4e0a-a6af-8edc478c2b84, 8, Finished, Available, Finished)

In [9]:
Max_ID=spark.sql("select coalesce(max(ShipMode_ID),0) from ecom_proj_LH.gold_shipmode").first()[0]

StatementMeta(, 351a858d-7f20-4e0a-a6af-8edc478c2b84, 11, Finished, Available, Finished)

In [10]:
df_final=df_bronze.withColumn("ShipMode_ID",monotonically_increasing_id()+Max_ID+1)
df_final.createOrReplaceTempView("ViewShipMode")

StatementMeta(, 351a858d-7f20-4e0a-a6af-8edc478c2b84, 12, Finished, Available, Finished)

In [12]:
%%sql
merge into ecom_proj_LH.gold_shipmode as gs
using ViewShipMode as vs
on gs.Ship_Mode=vs.Ship_Mode
when matched then 
update set
Modified_TS = current_timestamp()

when not matched then 
INSERT
(
    gs.ShipMode_ID,
    gs.Ship_Mode,
    gs.Created_TS,
    gs.Modified_TS
)
VALUES
(
    vs.ShipMode_ID,
    vs.Ship_Mode,
    current_timestamp(),
    current_timestamp()
)

StatementMeta(, 351a858d-7f20-4e0a-a6af-8edc478c2b84, 14, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>